In [ ]:
!pip install wandb
!pip install fairlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 KB 8.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, preprocessing
from datetime import datetime, timedelta

import wandb
import random

from fairlearn.postprocessing import ThresholdOptimizer, plot_threshold_optimizer
from fairlearn.metrics import demographic_parity_ratio, equalized_odds_ratio
from fairlearn.reductions import DemographicParity


from google.colab import drive

from sklearn.model_selection import KFold, cross_val_score



drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [155]:
import torch
from torch import nn
from torch.nn import functional as F


class ConstraintLoss(nn.Module):
    def __init__(self, n_class=2, alpha=1, p_norm=2):
        super(ConstraintLoss, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.alpha = alpha
        self.p_norm = p_norm
        self.n_class = n_class
        self.n_constraints = 2
        self.dim_condition = self.n_class + 1
        self.M = torch.zeros((self.n_constraints, self.dim_condition))
        self.c = torch.zeros(self.n_constraints)

    def mu_f(self, X=None, y=None, sensitive=None):
        return torch.zeros(self.n_constraints)

    def forward(self, X, out, sensitive, y=None):
        # Reshapes sensitive attribute tensor to the same shape as the output 
        sensitive = sensitive.view(out.shape)
        # Reshapes y (label) tensor to the same shape as the output 
        if isinstance(y, torch.Tensor):
            y = y.view(out.shape)
        # Get probability output by applying sigmoid (like logistic regression?)
        out = torch.sigmoid(out)
        # Get the mu_f value given these tensors 
        mu = self.mu_f(X=X, out=out, sensitive=sensitive, y=y)
        # Gap constraint???
        # Apply relu to matrix vector product of M compared to mu-c 
        # Using cuda if applicable (self.device references)
        gap_constraint = F.relu(
            torch.mv(self.M.to(self.device), mu.to(self.device)) - self.c.to(self.device)
        )
        # Using the L2 Norm of the gap constraint as regularizer with alpha parameter. 
        if self.p_norm == 2:
            cons = self.alpha * torch.dot(gap_constraint, gap_constraint)
        else:
            cons = self.alpha * torch.dot(gap_constraint.detach(), gap_constraint)
        return cons


class DemographicParityLoss(ConstraintLoss):
    def __init__(self, sensitive_classes=[0, 1], alpha=1, p_norm=2):
        """loss of demograpfhic parity

        Args:
            sensitive_classes (list, optional): list of unique values of sensitive attribute. Defaults to [0, 1].
            alpha (int, optional): [description]. Defaults to 1.
            p_norm (int, optional): [description]. Defaults to 2.
        """
        self.sensitive_classes = sensitive_classes
        self.n_class = len(sensitive_classes)
        super(DemographicParityLoss, self).__init__(
            n_class=self.n_class, alpha=alpha, p_norm=p_norm
        )
        
        self.n_constraints = 2 * self.n_class
        self.dim_condition = self.n_class + 1
        self.M = torch.zeros((self.n_constraints, self.dim_condition))
        for i in range(self.n_constraints):
            j = i % 2
            if j == 0:
                self.M[i, j] = 1.0
                self.M[i, -1] = -1.0
            else:
                self.M[i, j - 1] = -1.0
                self.M[i, -1] = 1.0
            #print(self.M)
        self.c = torch.zeros(self.n_constraints)

    def mu_f(self, X, out, sensitive, y=None):
        expected_values_list = []
        for v in self.sensitive_classes:
            # Get the index for each of the senstive classes 
            idx_true = sensitive == v  # torch.bool
            # Get the average prediction for that sensitive class
            expected_values_list.append(out[idx_true].mean())
        # Append the overall mean to the expected values list
        print(expected_values_list) 
        expected_values_list.append(out.mean())
        return torch.stack(expected_values_list)

    def forward(self, X, out, sensitive, y=None):
        return super(DemographicParityLoss, self).forward(X, out, sensitive)

# Convex 
class EqualiedOddsLoss(ConstraintLoss):
    def __init__(self, sensitive_classes=[0, 1], alpha=1, p_norm=2):
        """loss of demograpfhic parity

        Args:
            sensitive_classes (list, optional): list of unique values of sensitive attribute. Defaults to [0, 1].
            alpha (int, optional): [description]. Defaults to 1.
            p_norm (int, optional): [description]. Defaults to 2.
        """
        self.sensitive_classes = sensitive_classes
        self.y_classes = [0, 1]
        self.n_class = len(sensitive_classes)
        self.n_y_class = len(self.y_classes)
        super(EqualiedOddsLoss, self).__init__(n_class=self.n_class, alpha=alpha, p_norm=p_norm)
        # K:  number of constraint : (|A| x |Y| x {+, -})
        self.n_constraints = self.n_class * self.n_y_class * 2
        # J : dim of conditions  : ((|A|+1) x |Y|)
        self.dim_condition = self.n_y_class * (self.n_class + 1)
        self.M = torch.zeros((self.n_constraints, self.dim_condition))
        # make M (K * J): (|A| x |Y| x {+, -})  *   (|A|+1) x |Y|) )
        self.c = torch.zeros(self.n_constraints)
        element_K_A = self.sensitive_classes + [None]
        for i_a, a_0 in enumerate(self.sensitive_classes):
            for i_y, y_0 in enumerate(self.y_classes):
                for i_s, s in enumerate([-1, 1]):
                    for j_y, y_1 in enumerate(self.y_classes):
                        for j_a, a_1 in enumerate(element_K_A):
                            i = i_a * (2 * self.n_y_class) + i_y * 2 + i_s
                            j = j_y + self.n_y_class * j_a
                            self.M[i, j] = self.__element_M(a_0, a_1, y_1, y_1, s)

    def __element_M(self, a0, a1, y0, y1, s):
        if a0 is None or a1 is None:
            x = y0 == y1
            return -1 * s * x
        else:
            x = (a0 == a1) & (y0 == y1)
            return s * float(x)

    def mu_f(self, X, out, sensitive, y):
        expected_values_list = []
        for u in self.sensitive_classes:
            for v in self.y_classes:
                idx_true = (y == v) * (sensitive == u)  # torch.bool
                expected_values_list.append(out[idx_true].mean())
        # sensitive is star
        for v in self.y_classes:
            idx_true = y == v
            expected_values_list.append(out[idx_true].mean())
        return torch.stack(expected_values_list)

    def forward(self, X, out, sensitive, y):
        return super(EqualiedOddsLoss, self).forward(X, out, sensitive, y=y)

In [ ]:
# https://towardsdatascience.com/logistic-regression-with-pytorch-3c8bbea594be
class LogisticRegression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         (self.linear) 

         outputs = torch.sigmoid(self.linear(x))
         return outputs

In [ ]:
def label_encode(df):

  for i in df.columns:
    tmp = df[i].iloc[0]
    
    if type(tmp) != int or type(tmp) != float:
      le = preprocessing.LabelEncoder()
      df[i] = le.fit_transform(df[i].values)

  return df

def one_hot(df):
  
  for i in df.columns:
    tmp = df[i].iloc[0]

    if i == 'Loan_Status':
      pass
    elif type(tmp) == str:
      df = pd.get_dummies(df, columns = [i])
      # ohe = preprocessing.OneHotEncoder()
      # df[i] =  ohe.fit_transform(df[i].values.reshape(-1, 1)) # double check the shape

  return df


def get_dataset(name):
  if name == 'compas':
    compas_link ='https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv'
    df = pd.read_csv(compas_link, on_bad_lines='skip')
    df = label_encode(df)

  else: 
    df = pd.read_csv("/content/drive/MyDrive/Project/Data/adult.csv")
    df = label_encode(df)

  return df

def split(x, y, sensitive_features, train_ratio, test_ratio):
  validation_ratio = 100 - train_ratio - test_ratio

  if validation_ratio < 0:
    print ("Incorrect Ratios")
    return -1

  # train is now 75% of the entire data set
  x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(x, y, sensitive_features, test_size=1 - train_ratio, random_state=42)

  # test is now 10% of the initial data set
  # validation is now 15% of the initial data set
  x_val, x_test, y_val, y_test, a_val, a_test = train_test_split(x_test, y_test, a_test, test_size=test_ratio, random_state=42) 

  return  x_train, x_test, x_val, y_train, y_test, y_val, a_train, a_test, a_val

def mae(prediction, true):
  return metrics.mean_absolute_error(prediction, true)

def accuracy(prediction, true):
  true = true.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)

  return (metrics.accuracy_score(prediction, true))

def precision(prediction, true):
  true = true.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)
  return (metrics.precision_score(prediction, true))

def recall(prediction, true):
  true = true.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)
  return (metrics.recall_score(prediction, true))

def tpr(prediction, true):
  return np.logical_and(prediction == 1, true== 1).sum()/prediction.shape[0]

def fpr(prediction, true):
  return np.logical_and(prediction == 1, true== 0).sum()/prediction.shape[0]

def tnr(prediction, true):
  return np.logical_and(prediction == 0, true== 0).sum()/prediction.shape[0]

def fnr(prediction, true):
  return np.logical_and(prediction == 0, true== 1).sum()/prediction.shape[0]


def dpr(prediction, true, sensitive_features):
  return demographic_parity_ratio(prediction, true, sensitive_features=sensitive_features)

def eor(prediction, true, sensitive_features):
  return equalized_odds_ratio(prediction, true, sensitive_features=sensitive_features)
  

def test(model, x_test, y_test):
  x_test, y_test = torch.tensor(x_test), torch.tensor(y_test)
  outputs = model(x_test.float()) 
  loss = criterion(outputs, y_test.float())

  # log model results
  wandb.log({"test_loss": loss.item(), 
          "test_accuracy": accuracy(outputs, y_test), 
          "test_precision": precision(outputs, y_test), 
          "test_recall": recall(outputs, y_test)})
  wandb.watch(model)
  print ('Test Results, Loss: {:.4f},  Accuracy: {:.4f},  Precision: {:.4f},  Recall: {:.4f}' 
                  .format(loss.item(), 100 * accuracy(outputs, y_test),precision(outputs, y_test), recall(outputs, y_test)))



def train( model, criterion, optimizer, name,   lr,   x_train, x_val, y_train, y_val, a_train, a_val,  alpha = None, regularizers = None, num_epochs = None):
  x_train, x_val, y_train, y_val, a_train, a_val = torch.from_numpy(x_train), torch.from_numpy(x_val), torch.from_numpy(y_train), torch.from_numpy(y_val), torch.from_numpy(a_train), torch.from_numpy(a_val)

  config= {
      "learning_rate":lr,
      "epochs": num_epochs,
      "model":model.__class__.__name__,
      "criterion":criterion.__class__.__name__,
      "optimizer":optimizer.__class__.__name__,
      "train_ratio":train_ratio, 
      "test_ratio":test_ratio,
      "data":name,
    }
  
  if regularizers != None:
    for i in range(len(regularizers)):
      config['regularizer_'+str(i)] = regularizers[i]
      config['alpha_'+str(i)] = alpha[i]

  # # store the hyperparameters in weights and bias
  wandb.init(project=name, entity="mie424",config = config)

  epoch_loss_train = []
  epoch_accuracy_train = []
  epoch_precision_train = []
  epoch_recall_train = []

  epoch_loss_val = []
  epoch_accuracy_val = []
  epoch_precision_val = []
  epoch_recall_val = []

  # train loop
  for epoch in range(num_epochs):

    outputs = []
    # for i in range(0, x_train.shape[0], 10):
    optimizer.zero_grad()
  
    # x_i = x_train[i].float()

    # Forward pass
    outputs   = model(x_train.float()) 

      
        
    # outside of lop
    loss = criterion(outputs, y_train.float())
    if regularizers != None:
      # loss function
      for i in range(len(regularizers)):
        loss += regularizers[i](x_train.float(), outputs, a_train.float(), y_train.float()) #*alpha[i]
    # Backward and optimize
    
    loss.backward()
    optimizer.step()

    
    # store the epoch
    epoch_loss_train += [loss.item()]
    epoch_accuracy_train += [accuracy(outputs, y_train)]
    epoch_precision_train += [precision(outputs, y_train)]
    epoch_recall_train += [recall(outputs, y_train)]

    # log model results
    wandb.log({"train_loss": epoch_loss_train[-1], 
              "train_accuracy": epoch_accuracy_train[-1], 
              "train_precision": epoch_precision_train[-1], 
              "train_recall": epoch_recall_train[-1], })
    
    wandb.watch(model)

    # print results
    print ('Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.4f},  Precision: {:.4f},  Recall: {:.4f}' 
                  .format(epoch+1, num_epochs, epoch_loss_train[-1], 100 * epoch_accuracy_train[-1],epoch_precision_train[-1],epoch_recall_train[-1]))
    
    # Validation loop
    with torch.no_grad():
      
      # model results
      outputs_v =  model(x_val.float()) 

      loss_v = criterion(outputs_v, y_val.float())
      if regularizers != None:
        # loss function
        for i in range(len(regularizers)):
          loss_v += regularizers[i](x_val.float(), outputs_v, a_val.float(), y_val.float()) #* alpha[i]
      

    
      # store resulst for the pass through
      epoch_loss_val += [loss_v.item()]
      epoch_accuracy_val += [accuracy(outputs_v, y_val)]
      epoch_precision_val += [precision(outputs_v, y_val)]
      epoch_recall_val += [recall(outputs_v, y_val)]
      
      # log and print model results
      wandb.log({"val_loss": epoch_loss_val[-1], 
                  "val_accuracy": epoch_accuracy_val[-1], 
                  "val_precision": epoch_precision_val[-1], 
                  "val_recall": epoch_recall_val[-1], })
      wandb.watch(model)

      print('Accuracy of validation : {} % ,  Loss: {:.4f},  Precision: {:.4f},  Recall: {:.4f}'.format(100 * epoch_accuracy_val[-1], epoch_loss_val[-1],epoch_precision_val[-1], epoch_recall_val[-1])) 



In [156]:
wandb.login()
# get df
name = "adult"

df = get_dataset(name)

# x_var, y_var, sensitive  = ['race', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'v_decile_score', 'v_score_text'], ['two_year_recid'], ['race']

      #                        'sex', 'dob',
      #  'age',  'race', 'juv_fel_count', 'decile_score',
      #  'juv_misd_count', 'juv_other_count', 'priors_count',
      #  'days_b_screening_arrest',  'c_days_from_compas',
      #  'c_charge_degree', 'c_charge_desc',  'r_case_number',
      #  'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
      #  'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
      #  'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
      #  'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
      #  'v_type_of_assessment', 'v_decile_score', 'v_score_text',
      #   'in_custody', 'out_custody'],

x_var, y_var, sensitive = ['age', 'work_class', 'education', 'education_num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex','capital_gain', 'capital_loss',
       
        'hours_per_week', 
       ]       , ['income'], ['race']
# 'Loan_Amount_Term','LoanAmount',


train_ratio, test_ratio = 0.7, 0.66

# select x, y, and sensitive features
x = df[x_var].values 
y = df[y_var].values
sensitive_features = df[sensitive].values


# split 
x_train, x_test, x_val, y_train, y_test, y_val, a_train, a_test, a_val = split(x, y, sensitive_features, train_ratio, test_ratio)


num_epochs = 200

alpha_dp = 1
DemographicParity = DemographicParityLoss(sensitive_classes=[0, 1], alpha=alpha_dp) # constraint 
alpha_eo = 1
EqualizedOdds = EqualiedOddsLoss(sensitive_classes=[0, 1], alpha=alpha_eo) # constraint 


alphas = [alpha_dp,alpha_eo]
regularizers = [DemographicParity,EqualizedOdds]
input_dim =  x_train.shape[1]# Two inputs x1 and x2 
output_dim = 1 # Single binary output 
lr = 0.001

model = LogisticRegression(input_dim,output_dim)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

train( model, 
      criterion=criterion, 
      optimizer=optimizer, 
      name=name, 
      lr=lr, 
      x_train=x_train, 
      x_val=x_val, 
      y_train=y_train, 
      y_val=y_val, 
      a_train=a_train, 
      a_val=a_val,  
      alpha = alphas, 
      regularizers = regularizers, 
      num_epochs = num_epochs)

test(model, x_test, y_test)
wandb.finish()


[tensor(0.7218, grad_fn=<MeanBackward0>), tensor(0.7239, grad_fn=<MeanBackward0>)]
Epoch [1/200], Loss: 7.1189,  Accuracy: 26.5532,  Precision: 0.9879,  Recall: 0.2467
[tensor(0.7230), tensor(0.7195)]
Accuracy of validation : 27.883167720566092 % ,  Loss: 5.7890,  Precision: 0.9813,  Recall: 0.2490
[tensor(0.7143, grad_fn=<MeanBackward0>), tensor(0.7175, grad_fn=<MeanBackward0>)]
Epoch [2/200], Loss: 5.7761,  Accuracy: 27.9616,  Precision: 0.9796,  Recall: 0.2493
[tensor(0.7132), tensor(0.7071)]
Accuracy of validation : 29.84040951520626 % ,  Loss: 4.7123,  Precision: 0.9515,  Recall: 0.2503
[tensor(0.6997, grad_fn=<MeanBackward0>), tensor(0.7053, grad_fn=<MeanBackward0>)]
Epoch [3/200], Loss: 4.6074,  Accuracy: 30.5765,  Precision: 0.9504,  Recall: 0.2525
[tensor(0.6964), tensor(0.6854)]
Accuracy of validation : 34.23667570009034 % ,  Loss: 3.7760,  Precision: 0.8818,  Recall: 0.2534
[tensor(0.6777, grad_fn=<MeanBackward0>), tensor(0.6868, grad_fn=<MeanBackward0>)]
Epoch [4/200], Loss

test_accuracy,▁
test_loss,▁
test_precision,▁
test_recall,▁
train_accuracy,▁▃▅▅▆▆▆▆▇▇▇▇▇▇██████████████████████████
train_loss,█▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_precision,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_recall,▂▁▁▁▂▂▂▂▃▃▃▄▄▅▅▆▆▇▇▇▇▇██████████████████
val_accuracy,▁▃▅▅▆▆▆▆▇▇▇▇▇▇▇█████████████████████████
val_loss,█▄▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [ ]:
len(x_train)/(len(x_test)+len(x_val)+len(x_train))

0.6999785018887626

In [ ]:
len(x_val)

7815

In [ ]:
len(x_train)

22792